In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [2]:
# In this notepad we want to explore datatypes in our raw files
# and create an empty Postgresql table in remote database

In [3]:
# Let's load environmental vairables and define paths
load_dotenv(dotenv_path='../.env')
DATABASE_URL = os.getenv('DATABASE_URL')
DATABASE_URL = DATABASE_URL.replace('postgres:', 'postgresql:') # This stpe is requred for sqlalchemy to work properly
print(os.listdir('../airflow/data'))

['extracted_2022_09.parquet', 'extracted_2022_09.csv']


In [4]:
# Read parquet file to dataframe
file_path = '../airflow/data/extracted_2022_09.parquet'
df = pd.read_parquet(file_path)

In [5]:
df.head()

,OBJECTID,ACRES,SITE_ID,LOCATION,WATERSHED,SIGNATURES,BED_DENSITY,FIELD_06_V,FIELD_02_V,SHAPE_Length,SHAPE_Area
0,1,6.96160,133,Rocky Neck State Park area,12,1,M,Y,N,0.012991,3.028277e-06
1,2,5.00563,134,Rocky Neck State Park area,12,1,M,Y,N,0.007813,2.177570e-06
2,3,4.63277,135,Rocky Neck State Park area,12,1,M,Y,N,0.006846,2.015533e-06
3,4,1.37520,136,Rocky Neck State Park area,12,1,M,not field checked,N,0.004425,5.982568e-07
4,5,1.43834,67,Mumford Cove area,6,1,M,"Y,N",Y,0.004711,6.259592e-07


In [6]:
# Create engine to connect to DB
engine = create_engine(DATABASE_URL)

In [7]:
# Check schema suggested by pandas
# It's not ideal but it shold work
print(pd.io.sql.get_schema(df, name='extracted_data', con=engine))


CREATE TABLE extracted_data (
	"OBJECTID" BIGINT, 
	"ACRES" FLOAT(53), 
	"SITE_ID" BIGINT, 
	"LOCATION" TEXT, 
	"WATERSHED" BIGINT, 
	"SIGNATURES" BIGINT, 
	"BED_DENSITY" TEXT, 
	"FIELD_06_V" TEXT, 
	"FIELD_02_V" TEXT, 
	"SHAPE_Length" FLOAT(53), 
	"SHAPE_Area" FLOAT(53)
)




In [8]:
#df.dtypes

In [9]:
# Create table in postgresql
df.head(0).to_sql(name='extracted_data', con=engine, if_exists='replace')

0

In [10]:
# Close connection to DB
engine.dispose()